<a href="https://colab.research.google.com/github/IlyaS-26/Telecom/blob/main/lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лабораторная работа №7
Сергеев Илья, группа 3530901/90101

In [ ]:
import os

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

PI2 = np.pi * 2

Упражнение 7.1

Прогнаны примеры из блокнота chap07.ipynb.

Упражнение 7.2

Для реализации быстрого преобразования Фурье нужно разделить массив на чётные (e) и нечётные элементы (o); вычислить DFT e и o, делая рекурсивные вызовы; вычислить DFT для каждого значения n, используя Лемму Дэниеолсона-Ланцоша.

Для теста начнем с небольшого реального сигнала и вычислим его БПФ:

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Реализация ДПФ из книги:

In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

Можем подтвердить, что эта реализация дает тот же результат.

In [4]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

В качестве шага к созданию рекурсивного БПФ начнем с версии, которая разбивает входной массив и использует np.fft.fft для вычисления БПФ половин

In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Получаем те же результаты:

In [6]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Теперь мы можем заменить np.fft.fft рекурсивными вызовами и добавить базовый вариант:

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Получаем те же результаты:

In [8]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16